# 識別率の高いボクセルのブロードマンエリア特定

----  
  
引数：識別率一覧/ボクセル-ブロードマンエリア対応表が記録されたcsvファイルがあるディレクトリまでのパス  
  
---  
  
入力：識別率一覧 ACCURACY[(10cv or loo)]_Voxtimseries(時系列幅)_SVM.csv / ボクセル-ブロードマンエリア対応表 Voxel_Brodmann_Table.csv  
   
---
  
出力：ACCURACY_BA[交差検証法]_VOX(ベクトルの種類)_SVM.csv 
  
---  
  
各ボクセルがどのブロードマンエリアであるかを特定，識別率が高い順に並べ替える．  

In [13]:
from os.path import join as pathjoin
from pprint import pprint
# from nifti import NiftiImage
import glob
import numpy as np
import pandas as pd
import sys
import pickle
# import dill
import csv

# .pyで実行するときは%matplotlib inlineをコメントアウト！！！！
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
####### vecName,tsscan書き換え

# args = sys.argv
# PATH = args[1]

#jupyter notebookのときはここで指定
PATH = '../State-2fe_MaskBrodmann/'

# ベクトルの種類
vecName = 'univariate'

# 時系列幅
tsscan = '45'

if vecName != 'timeseries':
    
    tsscan = ''

Subject = ['20181029rn', '20181029su', '20181029tm']

# 交差検証法の種類
cv = '10cv'

# 機械学習アルゴリズム
ML = 'SVM'

# 各被験者ごとのRawDataディレクトリまでのパス

PATH_raw = []

for subName in Subject:
    
    PATH_raw = PATH_raw + [PATH + subName + '/mb/RawData/']

In [15]:
for sub in PATH_raw:
    
    print(sub)
    
    ####### ACMIDのcsvファイルを読み込む場合 ########
    
    # 識別率一覧を記録したMID（途中経過）csvファイルへのパス
#     PATH_ac = []

#     for name in glob.glob(sub + 'ACMID*'):
#         PATH_ac = PATH_ac + [name]
        
#     # 交差検証法によって抽出
#     PATH_ac = [s for s in PATH_ac if cv in s]
    
#         # 識別率一覧を読み込み

#     ac = pd.DataFrame(index = [], columns = [])

#     for file in PATH_ac:

#         acmid = pd.read_csv(file, header = 0)
#         ac = pd.concat([ac, acmid])

    # 識別率一覧を記録したACCURACYcsvファイルへのパス
    PATH_ac = sub + 'ACCURACY[' + cv + ']_VOX' + vecName + tsscan + '_' + ML + '.csv'
    
    # 識別率一覧を読み込み
    ac = pd.read_csv(PATH_ac, header = 0)


    # ボクセル-ブロードマンエリア対応表を記録したファイルへのパス
    PATH_tab = sub + 'VoxelBrodmannTable.csv'


    ac.columns = ['BAvoxelNum', 'accuracy(%)']

    # ボクセル-ブロードマンエリア対応表を読み込み
    tab = pd.read_csv(PATH_tab, header = 0)

    # acとtabをマージすることで各ボクセルのブロードマンエリアを特定
    AcTab = pd.merge(ac, tab, on = ['BAvoxelNum'], how = 'left')

    # accuracyでソートすることで識別率の高い順に並べる
    # ascending = Falseとすることで昇順に
    AcTab = AcTab.sort_values('accuracy(%)', ascending = False)
    
    
    # ソート後の順番（ランキング）を取得する
    
    # インデックスを振り直す前のカラム名を取得しておく
    before = AcTab.columns
    
    # インデックスを振り直した後のカラム名を作成する
    after = ['Rank'] + list(before)
    
    # インデックスを振り直す
    AcTab = AcTab.reset_index(drop = True)
    
    # 振り直すことで得た連番のインデックスを列として使いする
    AcTab = AcTab.reset_index()
    
    # 新しいカラム名をつけておく
    AcTab.columns = after
    
    # 連番は0から始まっているので1から始まるように+1しておく
    AcTab['Rank'] = AcTab['Rank'] + 1
    
    
    # BAvoxelNumとaccuracyを行名に設定することで(MultiIndex)重複を分かりやすくする
    # (一つのボクセルに対して複数のブロードマンエリアが対応していることがある）
    AcTab = AcTab.set_index(['BAvoxelNum', 'accuracy(%)'])

    
    PATH_csv = sub + 'ACCURACY_BA' + '[' + cv + ']_VOX' + vecName + tsscan + '_' + ML + '.csv'
    print(PATH_csv)
    AcTab.to_csv(PATH_csv)

../State-2fe_MaskBrodmann/20181029rn/mb/RawData/
../State-2fe_MaskBrodmann/20181029rn/mb/RawData/ACCURACY_BA[10cv]_VOXunivariate_SVM.csv
../State-2fe_MaskBrodmann/20181029su/mb/RawData/
../State-2fe_MaskBrodmann/20181029su/mb/RawData/ACCURACY_BA[10cv]_VOXunivariate_SVM.csv
../State-2fe_MaskBrodmann/20181029tm/mb/RawData/
../State-2fe_MaskBrodmann/20181029tm/mb/RawData/ACCURACY_BA[10cv]_VOXunivariate_SVM.csv


In [8]:
# looと10分割cvの比較

In [22]:
# loo = pd.read_csv('../MaskBrodmann2/20181119tm/RawData/AcVoxelBrodmannTable.csv', header = 0, index_col = None)

In [23]:
# cv10 = pd.read_csv('../MaskBrodmann2/20181119tm/RawData/AcVoxelBrodmannTable10cv.csv', header = 0, index_col = None)

In [25]:
# pd.merge(loo, cv10, on = ['BAvoxelNum'], how = 'right')

,BAvoxelNum,accuracy(%)_x,x_x,y_x,z_x,BrodmannArea_x,accuracy(%)_y,x_y,y_y,z_y,BrodmannArea_y
0,Voxel194,76.9,5.0,44.0,39.0,BrodmannArea42,73.0,5,44,39,BrodmannArea42
1,Voxel12050,76.7,20.0,37.0,26.0,unknown,70.9,20,37,26,unknown
2,Voxel193,76.3,5.0,44.0,38.0,BrodmannArea42,71.7,5,44,38,BrodmannArea42
3,Voxel547,75.4,6.0,44.0,40.0,BrodmannArea42,70.4,6,44,40,BrodmannArea42
4,Voxel14491,75.4,23.0,35.0,31.0,BrodmannArea19,68.7,23,35,31,BrodmannArea19
5,Voxel6391,75.0,14.0,25.0,25.0,BrodmannArea37,68.7,14,25,25,BrodmannArea37
6,Voxel186,75.0,5.0,43.0,39.0,BrodmannArea42,69.8,5,43,39,BrodmannArea42
7,Voxel5411,74.8,13.0,25.0,25.0,BrodmannArea37,69.4,13,25,25,BrodmannArea37
8,Voxel22330,74.8,31.0,5.0,44.0,unknown,68.3,31,5,44,unknown
9,Voxel4421,74.6,12.0,25.0,25.0,BrodmannArea37,68.5,12,25,25,BrodmannArea37
